# DATA WRANGLING – LIMPIEZA Y PREPARACIÓN DEL DATASET  
### Proyecto Final – Minería de Datos  
### Archivo base: retail_sales_dataset.csv

En esta sección realizo el proceso de **Data Wrangling** (limpieza y preparación de datos)  
utilizando exclusivamente el dataset del proyecto:

**retail_sales_dataset.csv**

Este procedimiento está inspirado en lo trabajado en clase en los notebooks:

- `Data_Wrangling.ipynb`
- `Limpieza_y_Preparacion_de_Datos_Dataset_de_Viviendas_09_10_2025.ipynb`

El objetivo de esta etapa es dejar un dataset limpio y consistente, listo para:

- 2.3 Transformaciones  
- 2.4 Normalización  
- 2.5 Análisis Exploratorio (EDA)  
- 2.6 y 2.7 Modelos de Machine Learning  

Al final de esta sección se generará un nuevo archivo:

**`retail_sales_limpio.csv`**

que será la base oficial para las siguientes fases del proyecto.


In [1]:
# Importación de librerías necesarias para el Wrangling

import pandas as pd
import numpy as np


## Lectura del dataset original

En este paso cargo el archivo original del proyecto:

**`retail_sales_dataset.csv`**

que contiene la información de 1000 ventas en una tienda retail con las columnas:

- ID_Transaccion  
- Fecha  
- ID_Cliente  
- Genero  
- Edad  
- Categoria_Producto  
- Cantidad  
- Precio_Unitario  
- Monto_Total  

A partir de aquí, todas las operaciones se realizan con Pandas,  
siguiendo el enfoque visto en `Data_Wrangling.ipynb`.


In [2]:
# Lectura del dataset original

df = pd.read_csv('retail_sales_dataset.csv')

# Vista general para confirmar que el dataset se cargó correctamente
df.head()


,ID_Transaccion,Fecha,ID_Cliente,Genero,Edad,Categoria_Producto,Cantidad,Precio_Unitario,Monto_Total
0,1,24/11/2023,CUST001,Male,34,Beauty,3,50,150
1,2,27/02/2023,CUST002,Female,26,Clothing,2,500,1000
2,3,13/01/2023,CUST003,Male,50,Electronics,1,30,30
3,4,21/05/2023,CUST004,Male,37,Clothing,1,500,500
4,5,06/05/2023,CUST005,Male,30,Beauty,2,50,100


## Revisión rápida de la estructura del DataFrame

Antes de limpiar, hago una revisión rápida similar a la de la inspección inicial:

- Primeros registros  
- Dimensiones (filas, columnas)  
- Tipos de datos  

Aunque parte de esto ya se vio en el notebook 2.1, aquí lo repito brevemente  
para documentar el flujo completo dentro de la fase de Wrangling.


In [3]:
# Dimensiones del dataset
df.shape


(1000, 9)

In [4]:
# Dimensiones del dataset
df.shape


(1000, 9)

## Estandarización de nombres de columnas

Para facilitar el trabajo con el dataset y seguir buenas prácticas,  
convierto los nombres de las columnas a minúsculas y reemplazo espacios por guiones bajos.

De esta forma, las columnas quedan con este formato:

- id_transaccion  
- fecha  
- id_cliente  
- genero  
- edad  
- categoria_producto  
- cantidad  
- precio_unitario  
- monto_total  

Este paso es equivalente al renombrado que hicimos en `Data_Wrangling.ipynb`.


In [5]:
# Renombrar columnas a formato estandarizado

df.columns = df.columns.str.lower().str.replace(' ', '_')

# Verificación del cambio de nombres
df.columns


Index(['id_transaccion', 'fecha', 'id_cliente', 'genero', 'edad',
       'categoria_producto', 'cantidad', 'precio_unitario', 'monto_total'],
      dtype='object')

## Conversión de tipos de datos (columna fecha)

En el archivo CSV original, la columna `fecha` viene como texto (`object`).  
Para poder hacer análisis por año, mes o día, es necesario convertirla al tipo:

`datetime64[ns]`

Este mismo tipo de conversión se hizo en los ejemplos de viviendas en `Data_Wrangling.ipynb`.


In [6]:
# Conversión de la columna 'fecha' a formato datetime

df['fecha'] = pd.to_datetime(df['fecha'], errors='coerce')

# Verificar nuevamente los tipos de datos
df.dtypes


C:\Users\arion\AppData\Local\Temp\ipykernel_14972\4242823221.py:3: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['fecha'] = pd.to_datetime(df['fecha'], errors='coerce')


id_transaccion                 int64
fecha                 datetime64[ns]
id_cliente                    object
genero                        object
edad                           int64
categoria_producto            object
cantidad                       int64
precio_unitario                int64
monto_total                    int64
dtype: object

## Revisión de valores nulos

Como parte del Wrangling, verifico si existen valores nulos en alguna columna.

En clase usamos:

- `isna()` / `isnull()`  
- `sum()` para contar cuántos nulos hay por columna  

Aquí aplico exactamente el mismo enfoque pero sobre el dataset retail.


In [7]:
# Conteo de valores nulos por columna

df.isna().sum()


id_transaccion        0
fecha                 0
id_cliente            0
genero                0
edad                  0
categoria_producto    0
cantidad              0
precio_unitario       0
monto_total           0
dtype: int64

## Tratamiento de valores nulos en columnas numéricas (si existieran)

Aunque el dataset de Retail probablemente no contiene nulos,  
dejo documentado cómo se resolvería en caso de encontrarlos.

En particular, se aplicarían rellenos con la media para las columnas numéricas:

- edad  
- cantidad  
- precio_unitario  
- monto_total  

En caso de no haber nulos, el código no modificará los datos,  
pero sigue siendo importante mostrar el procedimiento.


In [8]:
# Definir las columnas numéricas clave

columnas_numericas = ['edad', 'cantidad', 'precio_unitario', 'monto_total']

# Rellenar valores nulos con la media de cada columna (solo si existieran nulos)

df[columnas_numericas] = df[columnas_numericas].fillna(df[columnas_numericas].mean())

# Verificar nuevamente valores nulos en estas columnas
df[columnas_numericas].isna().sum()


edad               0
cantidad           0
precio_unitario    0
monto_total        0
dtype: int64

## Detección y eliminación de registros duplicados

Otro paso importante en el Wrangling es verificar si existen filas duplicadas.  
Esto puede suceder cuando se integran varias fuentes o se registran ventas repetidas.

El procedimiento es:

1. Contar cuántas filas duplicadas existen  
2. Eliminar esos registros  
3. Confirmar que ya no haya duplicados  

Este enfoque viene directamente del notebook `Data_Wrangling.ipynb`.


In [9]:
# Conteo inicial de filas duplicadas

duplicados_iniciales = df.duplicated().sum()
duplicados_iniciales


np.int64(0)

In [10]:
# Eliminación de filas duplicadas (si hubiera)

df = df.drop_duplicates()

# Verificación después de eliminar duplicados
df.duplicated().sum()


np.int64(0)

In [11]:
# Nuevas dimensiones del dataset después de la limpieza

df.shape


(1000, 9)

## Filtros básicos y creación de subconjuntos de datos

En clase practicamos varios tipos de filtros con Pandas  
(filtrar por condiciones, por rangos y por pertenencia a un conjunto).

Aplico las mismas ideas sobre el dataset retail, por ejemplo:

- Filtrar ventas realizadas por clientes de cierto rango de edad  
- Filtrar solo cierto género  
- Filtrar una categoría específica de producto  


In [12]:
# Ejemplo 1: ventas realizadas por clientes menores de 30 años

df_menores_30 = df[df['edad'] < 30]
df_menores_30.head()


,id_transaccion,fecha,id_cliente,genero,edad,categoria_producto,cantidad,precio_unitario,monto_total
1,2,2023-02-27,CUST002,Female,26,Clothing,2,500,1000
10,11,2023-02-14,CUST011,Male,23,Clothing,2,50,100
12,13,2023-08-05,CUST013,Male,22,Electronics,3,500,1500
15,16,2023-02-17,CUST016,Male,19,Clothing,3,500,1500
16,17,2023-04-22,CUST017,Female,27,Clothing,4,25,100


In [13]:
# Ejemplo 2: ventas realizadas por clientes de 40 años o más

df_mayores_40 = df[df['edad'] >= 40]
df_mayores_40.head()


,id_transaccion,fecha,id_cliente,genero,edad,categoria_producto,cantidad,precio_unitario,monto_total
2,3,2023-01-13,CUST003,Male,50,Electronics,1,30,30
5,6,2023-04-25,CUST006,Female,45,Beauty,1,30,30
6,7,2023-03-13,CUST007,Male,46,Clothing,2,25,50
8,9,2023-12-13,CUST009,Male,63,Electronics,2,300,600
9,10,2023-10-07,CUST010,Female,52,Clothing,4,50,200


In [14]:
# Ejemplo 3: ventas de la categoría 'Electronics' (si existe en los datos)

df_electronics = df[df['categoria_producto'] == 'Electronics']
df_electronics.head()


,id_transaccion,fecha,id_cliente,genero,edad,categoria_producto,cantidad,precio_unitario,monto_total
2,3,2023-01-13,CUST003,Male,50,Electronics,1,30,30
7,8,2023-02-22,CUST008,Male,30,Electronics,4,25,100
8,9,2023-12-13,CUST009,Male,63,Electronics,2,300,600
12,13,2023-08-05,CUST013,Male,22,Electronics,3,500,1500
14,15,2023-01-16,CUST015,Female,42,Electronics,4,500,2000


In [15]:
# Ejemplo 4: ventas de un conjunto de categorías

categorias_objetivo = ['Clothing', 'Beauty']

df_categorias = df[df['categoria_producto'].isin(categorias_objetivo)]
df_categorias.head()


,id_transaccion,fecha,id_cliente,genero,edad,categoria_producto,cantidad,precio_unitario,monto_total
0,1,2023-11-24,CUST001,Male,34,Beauty,3,50,150
1,2,2023-02-27,CUST002,Female,26,Clothing,2,500,1000
3,4,2023-05-21,CUST004,Male,37,Clothing,1,500,500
4,5,2023-05-06,CUST005,Male,30,Beauty,2,50,100
5,6,2023-04-25,CUST006,Female,45,Beauty,1,30,30


## Ordenamiento de datos (sort_values)

Otra operación importante de Wrangling es ordenar los registros  
para identificar fácilmente los casos más relevantes.

Aquí ordeno las ventas por:

- Monto total de la compra (descendente)  
- Edad del cliente (ascendente)  


In [16]:
# Ordenar por monto_total de forma descendente

df_orden_monto = df.sort_values('monto_total', ascending=False)
df_orden_monto.head()


,id_transaccion,fecha,id_cliente,genero,edad,categoria_producto,cantidad,precio_unitario,monto_total
945,946,2023-05-08,CUST946,Male,62,Electronics,4,500,2000
71,72,2023-05-23,CUST072,Female,20,Electronics,4,500,2000
14,15,2023-01-16,CUST015,Female,42,Electronics,4,500,2000
576,577,2023-02-13,CUST577,Male,21,Beauty,4,500,2000
571,572,2023-04-20,CUST572,Male,31,Clothing,4,500,2000


In [17]:
# Ordenar por edad de forma ascendente

df_orden_edad = df.sort_values('edad', ascending=True)
df_orden_edad.head()


,id_transaccion,fecha,id_cliente,genero,edad,categoria_producto,cantidad,precio_unitario,monto_total
61,62,2023-12-27,CUST062,Male,18,Beauty,2,50,100
73,74,2023-11-22,CUST074,Female,18,Beauty,4,500,2000
594,595,2023-11-09,CUST595,Female,18,Clothing,4,500,2000
325,326,2023-09-15,CUST326,Female,18,Clothing,3,25,75
713,714,2023-02-12,CUST714,Female,18,Clothing,1,500,500


## Agrupaciones y agregaciones (groupby + agg)

En el notebook de viviendas hicimos muchas operaciones con `groupby` y `agg`  
para resumir información por grupos.

Aquí aplico el mismo enfoque al retail:

1. Ventas totales por categoría de producto  
2. Comportamiento de compra por género  
3. Estadísticas agregadas sobre el monto total  


In [18]:
# Ventas agregadas por categoría de producto

resumen_categoria = df.groupby('categoria_producto', as_index=False)['monto_total'].agg(
    ['count', 'sum', 'mean']
)

resumen_categoria


,categoria_producto,count,sum,mean
0,Beauty,307,143515,467.475570
1,Clothing,351,155580,443.247863
2,Electronics,342,156905,458.786550


In [19]:
# Ventas agregadas por género

resumen_genero = df.groupby('genero', as_index=False)['monto_total'].agg(
    ['count', 'sum', 'mean']
)

resumen_genero


,genero,count,sum,mean
0,Female,510,232840,456.549020
1,Male,490,223160,455.428571


## Estadísticas generales con `agg` sobre columnas numéricas

En clase utilizamos `agg` para ver mínimos, máximos, media, mediana, desviación estándar y varianza.

Aquí aplico algo similar sobre las columnas numéricas principales del dataset:

- edad  
- cantidad  
- precio_unitario  
- monto_total  


In [20]:
# Estadísticas descriptivas resumidas para columnas numéricas

estadisticas_numericas = df[['edad', 'cantidad', 'precio_unitario', 'monto_total']].agg(
    ['min', 'max', 'mean', 'median', 'std', 'var']
)

estadisticas_numericas


,edad,cantidad,precio_unitario,monto_total
min,18.000000,1.000000,25.000000,25.000000
max,64.000000,4.000000,500.000000,2000.000000
mean,41.392000,2.514000,179.890000,456.000000
median,42.000000,3.000000,50.000000,135.000000
std,13.681430,1.132734,189.681356,559.997632
var,187.181518,1.283087,35979.016917,313597.347347


## Creación del dataset limpio final

Después de:

- Renombrar columnas  
- Convertir `fecha` a datetime  
- Revisar y tratar valores nulos  
- Eliminar duplicados  
- Validar filtros y agregaciones  

el DataFrame `df` queda listo para ser utilizado como base en las siguientes fases.

A continuación guardo este DataFrame como:

**`retail_sales_limpio.csv`**

Este archivo será utilizado en:

- 2.3 Transformaciones  
- 2.4 Normalización  
- 2.5 EDA  
- Modelos de Machine Learning  


In [21]:
# Exportación del DataFrame limpio a un nuevo archivo CSV

df.to_csv('retail_sales_limpio.csv', index=False)

print("✅ Archivo 'retail_sales_limpio.csv' generado correctamente.")


✅ Archivo 'retail_sales_limpio.csv' generado correctamente.


## Conclusiones de la fase de Data Wrangling

En esta fase se logró:

- Estandarizar los nombres de las columnas  
- Convertir la columna `fecha` a tipo datetime  
- Verificar (y en su caso tratar) valores nulos en las columnas numéricas  
- Eliminar posibles registros duplicados  
- Aplicar filtros básicos para entender subconjuntos de interés  
- Ordenar la información por monto total y por edad  
- Generar resúmenes usando `groupby` y `agg`  
- Crear el archivo **`retail_sales_limpio.csv`** que servirá como base para las siguientes etapas

Con esto se cumple el punto **1.3.2 Data Wrangling** de la rúbrica del proyecto,  
documentando cada paso de la preparación y transformación inicial de los datos.
